In [1]:
import openai

In [2]:
client = openai.OpenAI(
    api_key="<token>",
    base_url="<url>" )

In [7]:
prompt = "Your assignment involves a two-step process:\nFirst, you will receive a task described in a single sentence. This task outlines what information you need to find or what question you need to answer. Read it properly.\nThen, you will create a SPARQL Query: Based on the task given, you will write a SPARQL query. SPARQL is a specialized query language used to retrieve and manipulate data stored in Resource Description Framework (RDF) format. Your goal is to craft a query that, when executed, fetches the data or answers required by the initial instruction.\nMake sure your SPARQL query is correctly formulated so that, upon execution, it produces the desired result matching the task's requirements.\nAnswer this following instruction: "

response = client.chat.completions.create(
    model="mistral",
    messages = [
        {
            "role": "user",
            "content": f"{prompt} \"Write a SPARQL query to list all the former and the current mayor of Paris from Wikidata.\""
        }
    ]
)

print(response.choices[0].message.content)

 To achieve the task of listing all the former and current mayors of Paris from Wikidata, we can construct the following SPARQL query. Please note that you'll need to have access to a Wikidata SPARQL endpoint like `https://query.wikidata.org/sparql` to execute this query.

```
PREFIX wd: <http://www.wikidata.org/entity/>
PREFIX wdt: <http://www.wikidata.org/prop/direct/>
PREFIX p: <http://wikidata.org/prop/>

SELECT DISTINCT ?mayor ?type
WHERE {
  # Define the subject, which is Paris
  BIND(wd:"Q131" AS ?paris)

  # Find all mayors of Paris using the P361 property, which denotes the office-holder position for an administrative territory.
  # We also include current mayor by using filter on active status.
  ?mayor wdt:P361/wdt:P279* wd:?id .
  FILTER(lang(label-en, ?mayor) = "Paris") # Filter for the English label containing Paris to avoid duplicates from translations
  BIND(STRAFTER(STRAFTER(str(?mayor), "Mayor of "), " ") AS ?current_mayor) # Extract the mayor's name
  OPTIONAL {
    